<a href="https://colab.research.google.com/github/backnotprop/Colab-Stable-Diffusion-2-Depth-UI/blob/main/%5BColab_Gradio%5D_stable_diffusion_2_depth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Stable Diffusion V2 Demo for the depth conditioning model (Depth2Img)

https://huggingface.co/stabilityai/stable-diffusion-2-depth

This stable-diffusion-2-depth model is resumed from stable-diffusion-2-base (512-base-ema.ckpt) and finetuned for 200k steps. Added an extra input channel to process the (relative) depth prediction produced by MiDaS (dpt_hybrid) which is used as an additional conditioning.

Notebook originally created by  [Ramos - backnotprop github]()


#### Note 
This notebook uses Gradio's `share` feature to serve the UI via a public URL, through a tunnel over HTTPs. To add security, this notebook forces you to set a `user/password` combo to access the UI 

### Required Setup



In [ ]:
import os
import time
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
from re import search

Use_xformers = True #@param {type:"boolean"}

if Use_xformers:
  with capture.capture_output() as cap:

    smi_out = getoutput('nvidia-smi')
    supported = search('(T4|P100|V100|A100|K80)', smi_out)

    if not supported:
      while True:
        print("\x1b[1;31mThe current GPU is not supported, try starting a new session.\x1b[0m")
        time.sleep(5)
    else:
      supported = supported.group(0)

    !pip install -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/{supported}/xformers-0.0.13.dev0-py3-none-any.whl

  clear_output()

In [ ]:
with capture.capture_output() as cap:
  !git clone --depth=1 https://github.com/Stability-AI/stablediffusion
  %cd /content/stablediffusion/
  !rm -rf /content/stablediffusion/.git/

clear_output()

In [ ]:
with capture.capture_output() as cap:
  !pip install -r requirements.txt
  !pip install transformers==4.19.2 diffusers invisible-watermark pytorch-lightning==1.8.3.post0 timm
  !pip install -e .
  
clear_output()

### Download models (512-depth, and midas)

In [ ]:
%cd /content/stablediffusion/

!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_hybrid-midas-501f0c75.pt
!wget https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt

!mkdir midas_models

!mv dpt_hybrid-midas-501f0c75.pt midas_models/


clear_output()

## Run with private gradio session

script likely wont start right away, and you will see standard `Setting up MemoryEfficientCrossAttention` output, and possible model downloads. 



In [ ]:
import fileinput
import sys

ui_username = "" #@param {type:"string"}
ui_password = "" #@param {type:"string"}

if ui_username == "" or ui_password == "":
  print("you must input a temporary username and password")

else:
  for line in fileinput.input('/content/stablediffusion/scripts/gradio/depth2img.py', inplace=True):
    if line.strip().startswith('block.launch()'):
        line = f'block.launch(share=True,auth=("{ui_username}", "{ui_password}"))\n'      
    sys.stdout.write(line)

  !python scripts/gradio/depth2img.py configs/stable-diffusion/v2-midas-inference.yaml /content/stablediffusion/512-depth-ema.ckpt
